# Example Optimiser

In [1]:
import os 
import sys
import json
sys.path.append(os.path.abspath(".."))

from py_modules.velopix_wrapper.parameter_optimisers import optimiserBase
from py_modules.velopix_wrapper.velopix_pipeline import Pipeline_TrackFollowing
from py_modules.velopix_wrapper.ReconstructionAlgorithms import ReconstructionAlgorithms

## Implement the optimiser child class

In [ ]:
from typing import Any, Dict
import random

class ExampleOptimiser(optimiserBase):
    def __init__(self, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.best_score = float("inf")

    def init(self) -> Dict[str, Any]:
        """
        Initializes the optimization process by setting an initial parameter map.
        """
        pMap = self._algorithm.get_config()  # Get a copy of the parameter schema

        for key, (expected_type, _) in pMap.items():
            if expected_type == float:
                pMap[key] = random.uniform(0, 1)  # Random float between 0 and 1
            elif expected_type == int:
                pMap[key] = random.randint(0, 10)  # Random integer between 0 and 10
            elif expected_type == bool:
                pMap[key] = random.choice([True, False])  # Random boolean
            elif expected_type == list:
                pMap[key] = []  # Assign an empty list (or populate it if needed)

        return pMap

    def next(self) -> Dict[str, Any]:
        """
        Generates the next parameter map by slightly modifying existing values.
        """
        next_pMap = self.current_config.copy()  # Create a copy to modify

        for key, (expected_type, _) in self._algorithm.get_config().items():
            if expected_type == float:
                # Apply a small random variation to float values
                next_pMap[key] = next_pMap.get(key, 0.5) * random.uniform(0.9, 1.1)
            elif expected_type == int:
                # Increase/decrease integers slightly (within range)
                next_pMap[key] = max(0, next_pMap.get(key, 5) + random.choice([-1, 0, 1]))
            elif expected_type == bool:
                # Flip a boolean value with a small probability
                next_pMap[key] = next_pMap.get(key, True) if random.random() > 0.1 else not next_pMap.get(key, True)
            elif expected_type == list:
                # Keep list as is (or modify if needed)
                next_pMap[key] = next_pMap.get(key, [])

        self.current_config = next_pMap  # Update the current config
        return self.current_config

    def objective_func(self) -> float:
        """
        Converts the results of an experiment into a numeric score.
        In this example, we simulate a loss function that we aim to minimize.
        """
        # Fake evaluation function
        last_run = self.runs[-1] # normaly we will take this to evaluate the paremeters
        return abs(self.learning_rate - 0.05) + random.uniform(0, 0.01)

    def is_finished(self) -> bool:
        """
        Determines if the optimization process is finished.
        In this case, it stops after `max_iterations` iterations.
        """
        return self.best_score < 0.3

**Load event data**

In [3]:
events = []
n_files = 100

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data)
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [4]:
pipeline = Pipeline_TrackFollowing(events=events, intra_node=False)

In [ ]:
Optimiser = ExampleOptimiser(learning_rate=0.1)
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=10) # DO NOT remove max_runs, chances are that this will run forever

Instantiating track_following solver with parameters
 max slopes: (0.04412785924218221, 0.07863575326277894)
 max tolerance: (0.6152999346192524, 0.39431315849870685)
 max scatter: 0.36127855443717405

Instantiating track_following solver with parameters
 max slopes: (0.4564881653538119, 0.536144008920365)
 max tolerance: (0.4508150155995477, 0.47057734349292935)
 max scatter: 0.5109007230639719

Instantiating track_following solver with parameters
 max slopes: (0.47555436611487284, 0.5372220435260123)
 max tolerance: (0.4096517729553249, 0.5138215655406297)
 max scatter: 0.5309262373969411

Instantiating track_following solver with parameters
 max slopes: (0.4697424217699221, 0.5319625842015002)
 max tolerance: (0.377332107445997, 0.513621871741172)
 max scatter: 0.5470608570680863

Instantiating track_following solver with parameters
 max slopes: (0.4536301282467597, 0.5583216552092701)
 max tolerance: (0.36679240739583585, 0.5015286271656776)
 max scatter: 0.561262221430136

Instant

In [ ]:
print(optimal_parameters) # Note these are just here for example...

{'x_slope': 0.5660441912208445, 'y_slope': 0.48542672255522284, 'x_tol': 0.3650939548201576, 'y_tol': 0.5242380583531047, 'scatter': 0.6681940041283005}
